# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f632dfa2df0>
├── 'a' --> tensor([[ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856]])
└── 'x' --> <FastTreeValue 0x7f632dfa2bb0>
    └── 'c' --> tensor([[ 1.5292, -0.1858,  0.2161,  1.1830],
                        [ 1.0340,  0.5212,  0.7626, -0.4958],
                        [ 2.2541,  1.2886,  1.5385,  0.9631]])

In [4]:
t.a

tensor([[ 0.5687,  0.6913,  0.3075],
        [-0.7871,  0.7048,  0.5856]])

In [5]:
%timeit t.a

82.9 ns ± 7.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f632dfa2df0>
├── 'a' --> tensor([[0.1412, 1.9152, 2.1684],
│                   [1.1060, 0.6144, 1.4247]])
└── 'x' --> <FastTreeValue 0x7f632dfa2bb0>
    └── 'c' --> tensor([[ 1.5292, -0.1858,  0.2161,  1.1830],
                        [ 1.0340,  0.5212,  0.7626, -0.4958],
                        [ 2.2541,  1.2886,  1.5385,  0.9631]])

In [7]:
%timeit t.a = new_value

68.4 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5687,  0.6913,  0.3075],
               [-0.7871,  0.7048,  0.5856]]),
    x: Batch(
           c: tensor([[ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5687,  0.6913,  0.3075],
        [-0.7871,  0.7048,  0.5856]])

In [11]:
%timeit b.a

58.8 ns ± 0.0673 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6865, -1.2531,  0.9640],
               [-0.8911, -0.0682, -0.0395]]),
    x: Batch(
           c: tensor([[ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631]]),
       ),
)

In [13]:
%timeit b.a = new_value

508 ns ± 0.108 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

806 ns ± 0.115 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 38.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 1.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f62560f7f40>
├── 'a' --> tensor([[[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]],
│           
│                   [[ 0.5687,  0.6913,  0.3075],
│                    [-0.7871,  0.7048,  0.5856]]])
└── 'x' --> <FastTreeValue 0x7f632dfa2c70>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 75.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6328f36c40>
├── 'a' --> tensor([[ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856],
│                   [ 0.5687,  0.6913,  0.3075],
│                   [-0.7871,  0.7048,  0.5856]])
└── 'x' --> <FastTreeValue 0x7f6328f36e80>
    └── 'c' --> tensor([[ 1.5292, -0.1858,  0.2161,  1.1830],
                        [ 1.0340,  0.5212,  0.7626, -0.4958],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 48.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.2 µs ± 61.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.5292, -0.1858,  0.2161,  1.1830],
                       [ 1.0340,  0.5212,  0.7626, -0.4958],
                       [ 2.2541,  1.2886,  1.5385,  0.9631]],
              
                      [[ 1.5292, -0.1858,  0.2161,  1.1830],
                       [ 1.0340,  0.5212,  0.7626, -0.4958],
                       [ 2.2541,  1.2886,  1.5385,  0.9631]],
              
                      [[ 1.5292, -0.1858,  0.2161,  1.1830],
                       [ 1.0340,  0.5212,  0.7626, -0.4958],
                       [ 2.2541,  1.2886,  1.5385,  0.9631]],
              
                      [[ 1.5292, -0.1858,  0.2161,  1.1830],
                       [ 1.0340,  0.5212,  0.7626, -0.4958],
                       [ 2.2541,  1.2886,  1.5385,  0.9631]],
              
                      [[ 1.5292, -0.1858,  0.2161,  1.1830],
                       [ 1.0340,  0.5212,  0.7626, -0.4958],
                       [ 2.2541,  1.2886,  1.5385,  0.9631]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631],
                      [ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631],
                      [ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631],
                      [ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631],
                      [ 1.5292, -0.1858,  0.2161,  1.1830],
                      [ 1.0340,  0.5212,  0.7626, -0.4958],
                      [ 2.2541,  1.2886,  1.5385,  0.9631],
                      [ 1.5292, -0.1858,  0.2161,  1.1830],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 63.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

342 µs ± 462 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
